# DS Lab - Final Assignment: Robust Journey Planning #

In [1]:
#Importations.
import pandas as pd
import glob
from geopy import distance
import numpy as np

import networkx as nx
import datetime
from datetime import timezone
import math

import trips
import schedules
import edges
import nodes

In [2]:
datetime.date(2017, 9, 30) + 1

TypeError: unsupported operand type(s) for +: 'datetime.date' and 'int'

In [2]:
# Reload dataframes
'''
df = pd.read_pickle('df')
info_trips = pd.read_pickle('info_trips')
visu_trips = pd.read_pickle('visu_trips')
df_schedules = pd.read_pickle('df_schedules')
df_edges = pd.read_pickle('df_edges')'''

In [3]:
# Define time slots
# WARNING those variable names should not be reassigned in the notebook
one = datetime.time(0, 0)
two = datetime.time(12, 0)
three = datetime.time(23, 59, 59)
defined_timeslots = [str(one) + '-' + str(two), str(two) + '-' + str(three)]

## Load the data

### Load the data published by the Open Data Platform Swiss Public Transport (https://opentransportdata.swiss).



In [4]:
#path on my computer of the files.
path = "2017-09/*.csv"

In [5]:
#useful list
data = []
#read each csv we had.
for fname in sorted(glob.glob(path)):
    d = pd.read_csv(fname, sep=';',low_memory=False)
    data.append(d)
    
#process the data from the csv files to a dataframe.
data = pd.concat(data)
data = pd.DataFrame(data)
data = data.reset_index()
data = data.drop(['index'], axis = 1)

In [6]:
#check the tail.
data.tail()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,FAELLT_AUS_TF,BPUIC,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
1776718,14.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504294,Vuiteboeuf,14.09.2017 05:57,NaN,UNBEKANNT,14.09.2017 05:57,NaN,UNBEKANNT,False
1776719,14.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504295,Baulmes,14.09.2017 06:01,NaN,UNBEKANNT,14.09.2017 06:01,NaN,UNBEKANNT,False
1776720,14.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504296,Six-Fontaines,14.09.2017 06:05,NaN,UNBEKANNT,14.09.2017 06:05,NaN,UNBEKANNT,False
1776721,14.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504297,Trois-Villes,14.09.2017 06:12,NaN,UNBEKANNT,14.09.2017 06:12,NaN,UNBEKANNT,False
1776722,14.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504298,Ste-Croix,14.09.2017 06:21,NaN,UNBEKANNT,NaN,NaN,PROGNOSE,False


### Load the station list data.

In [7]:
#path.
station_path = "BFKOORD_GEO"

In [8]:
station_file  = open(station_path, "r")

In [9]:
BPUIC = []
long = []
lat = []
name = []

for line in station_file:
    l = line.split()
    BPUIC.append(l[0])
    long.append(l[1])
    lat.append(l[2])
    name.append(l[5])


In [10]:
d = {'BPUIC':BPUIC,
     'longitude' : long,
     'latitude' : lat,
     'station name': name}
station_df = pd.DataFrame(d)
station_df = station_df[['BPUIC','longitude','latitude']]
station_df['BPUIC'] = station_df['BPUIC'].astype(np.int64)

In [11]:
station_df.head()

,BPUIC,longitude,latitude
0,2,26.074412,44.446770
1,3,1.811446,50.901549
2,4,1.075329,51.284212
3,5,-3.543547,50.729172
4,7,9.733756,46.922368


In [12]:
zurich_coord = (47.407665036,8.540331172)
salamanca = (40.96, -5.50)

In [13]:
print(distance.distance(zurich_coord, salamanca).km)

1328.6915399458765


In [14]:
df = pd.merge(data,station_df,on='BPUIC',how='left')

In [15]:
df.head()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF,longitude,latitude
0,13.09.2017,80:06____:17010:000,80:06____,DB,DB Regio AG,Zug,17010,RE,NaN,RE,...,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 05:45,13.09.2017 05:45:00,PROGNOSE,False,7.607284,47.568147
1,13.09.2017,80:06____:17012:000,80:06____,DB,DB Regio AG,Zug,17012,RE,NaN,RE,...,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 06:34,13.09.2017 06:34:00,PROGNOSE,False,7.607284,47.568147
2,13.09.2017,80:06____:17013:000,80:06____,DB,DB Regio AG,Zug,17013,RE,NaN,RE,...,Basel Bad Bf,13.09.2017 06:25,13.09.2017 06:28:00,PROGNOSE,NaN,NaN,PROGNOSE,False,7.607284,47.568147
3,13.09.2017,80:06____:17014:000,80:06____,DB,DB Regio AG,Zug,17014,RE,NaN,RE,...,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 09:48,13.09.2017 09:48:00,PROGNOSE,False,7.607284,47.568147
4,13.09.2017,80:06____:17015:000,80:06____,DB,DB Regio AG,Zug,17015,RE,NaN,RE,...,Basel Bad Bf,13.09.2017 08:06,13.09.2017 08:11:00,PROGNOSE,NaN,NaN,PROGNOSE,False,7.607284,47.568147


# Create Network

In [16]:
# transform a date to an integer. Dates within the same 24 hours
# interval map to the same integer. the interval is specied by the
# origin argument. 
# example origin = 2017.09.13 2h00
# date_to_integer(2017.09.13 14h05) returns 0
# date_to_integer(2017.09.14 1h28) returns 0
# date_to_integer(2017.09.14 8h38) returns 1
def date_to_integer(row, origin=datetime.datetime(2017, 9, 13, 2, 0, 0)):
    # set reference time in seconds
    origin_seconds = origin.replace(tzinfo=timezone.utc).timestamp()
    # transform datetime into seconds    
    try:
        # transform datetime into seconds
        # row['scheduled_departure_time'].
        curr_datetime_seconds = row[0].replace(tzinfo=timezone.utc).timestamp()
    except ValueError:
        try:
            # row['scheduled_arrival_time']
            curr_datetime_seconds = row[1].replace(tzinfo=timezone.utc).timestamp()
        except ValueError:
            print('Time information not found !')
    # set target
    target = curr_datetime_seconds - origin_seconds
    # set offset
    offset = 60*60*24
    # find integer value to associate with a day
    x = -2
    while not (x*offset <= target < (x+1)*offset):
        x += 1
    return x

In [17]:
# Renaming the columns of the dataframe
df.columns = ['trip_date', 'trip_id', 'operator_id', 'operator_abk', 
                'operator_name', 'product_id', 'line_id', 'line_text',
                'circulation_id', 'transport_text', 'additional_trip',
                'failed', 'stop_id', 'stop_name', 'scheduled_arrival_time',
                'actual_arrival_time', 'actual_arrival_time_status', 'scheduled_departure_time',
                'actual_departure_time',  'actual_departure_time_status', 'no_stop',
                'longitude', 'latitude']

In [18]:
# Convert columns 'trip_date', scheduled_arrival_time' and 'scheduled departure time' to datetime
df['trip_date'] = pd.to_datetime(df['trip_date'], format='%d.%m.%Y', errors='ignore')
df['scheduled_arrival_time'] = pd.to_datetime(df['scheduled_arrival_time'], format='%d.%m.%Y %H:%M', errors='ignore')
df['scheduled_departure_time'] = pd.to_datetime(df['scheduled_departure_time'], format='%d.%m.%Y %H:%M', errors='ignore')
df['actual_arrival_time'] = pd.to_datetime(df['actual_arrival_time'], format='%d.%m.%Y %H:%M:%S', errors='ignore')
df['actual_departure_time'] = pd.to_datetime(df['actual_departure_time'], format='%d.%m.%Y %H:%M:%S', errors='ignore')

In [19]:
# Limit number of trip_id to consider to save time
trip_ids = df['trip_id'].unique()[400:600]

In [20]:
info_trips = trips.get_trips(df, trip_ids)
info_trips = info_trips.reset_index(drop=True)
info_trips = info_trips.set_index(['operator_name', 'trip_id', 'product_id']).sort_index(axis=0, level=['trip_id'])
info_trips = info_trips[['date_id','stop_name', 'longitude', 'latitude', 'scheduled_departure_time', 'actual_departure_time', 'scheduled_arrival_time', 'actual_arrival_time']]

/home/mbanga/Epfl/LabsInDataScience/project_cff/trips.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  temp['date_id'] = temp[['scheduled_departure_time', 'scheduled_arrival_time']].apply(date_to_integer, axis=1)


In [21]:
info_trips.head()

date_id  \
operator_name                   trip_id         product_id            
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug             0.0   
                                                Zug             0.0   
                                                Zug             0.0   
                                                Zug             0.0   
                                                Zug             0.0   

                                                                   stop_name  \
operator_name                   trip_id         product_id                     
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug         Lancy-Pont-Rouge   
                                                Zug                   Genève   
                                                Zug            Les Tuileries   
                                                Zug         Genthod-Bellevue   
                                                Zug         Creux-de-Genthod   

                                                           longitude  \
operator_name                   trip_id         product_id             
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug         6.124948   
                                                Zug         6.142452   
                                                Zug         6.147340   
                                                Zug         6.153963   
                                                Zug         6.161320   

                                                             latitude  \
operator_name                   trip_id         product_id              
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug         46.185943   
                                                Zug         46.210203   
                                                Zug         46.249944   
                                                Zug         46.256731   
                                                Zug         46.263760   

                                                           scheduled_departure_time  \
operator_name                   trip_id         product_id                            
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug             2017-09-13 08:54:00   
                                                Zug             2017-09-13 09:02:00   
                                                Zug             2017-09-13 09:08:00   
                                                Zug             2017-09-13 09:10:00   
                                                Zug             2017-09-13 09:14:00   

                                                           actual_departure_time  \
operator_name                   trip_id         product_id                         
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug          2017-09-13 08:54:54   
                                                Zug          2017-09-13 09:05:13   
                                                Zug          2017-09-13 09:12:52   
                                                Zug          2017-09-13 09:12:52   
                                                Zug          2017-09-13 09:17:16   

                                                           scheduled_arrival_time  \
operator_name                   trip_id         product_id                          
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug                           NaT   
                                                Zug           2017-09-13 09:00:00   
                                                Zug           2017-09-13 09:08:00   
                                                Zug           2017-09-13 09:10:00   
                                                Zug           2017-09-13 09:13:00   

                                                           actual_arrival_time  
operator_name                   trip_id         product_id                      
Schweizerische Bundesbahnen

In [22]:
visu_trips = trips.visu_trips(info_trips.reset_index(), trip_ids)
visu_trips = visu_trips.set_index(['operator_name', 'trip_id', 'product_id']).sort_index(axis=0, level=['trip_id'])
visu_trips = visu_trips[['date_id', 'stop_list', 'scheduled_terminus_departure_time', 'scheduled_terminus_arrival_time']]

/home/mbanga/Epfl/LabsInDataScience/project_cff/trips.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  visu['stop_list'] = ' | '.join(visu['stop_name'])


In [23]:
visu_trips.head()

date_id  \
operator_name                   trip_id         product_id            
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug             0.0   
                                                Zug             1.0   
                                85:11:11032:001 Zug             0.0   
                                                Zug             1.0   
                                85:11:11033:001 Zug             0.0   

                                                                                                    stop_list  \
operator_name                   trip_id         product_id                                                      
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug         Lancy-Pont-Rouge | Genève | Les Tuileries | Ge...   
                                                Zug         Lancy-Pont-Rouge | Genève | Genève-Sécheron | ...   
                                85:11:11032:001 Zug         Coppet | Tannay | Mies | Pont-Céard | Versoix ...   
                                                Zug         Coppet | Tannay | Mies | Pont-Céard | Versoix ...   
                                85:11:11033:001 Zug         Lancy-Pont-Rouge | Genève | Les Tuileries | Ge...   

                                                           scheduled_terminus_departure_time  \
operator_name                   trip_id         product_id                                     
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug                      2017-09-13 08:54:00   
                                                Zug                      2017-09-14 08:54:00   
                                85:11:11032:001 Zug                      2017-09-13 09:03:00   
                                                Zug                      2017-09-14 09:03:00   
                                85:11:11033:001 Zug                      2017-09-13 09:22:00   

                                                           scheduled_terminus_arrival_time  
operator_name                   trip_id         product_id                                  
Schweizerische Bundesbahnen SBB 85:11:11031:001 Zug                    2017-09-13 09:25:00  
                                                Zug                    2017-09-14 09:25:00  
                                85:11:11032:001 Zug                    2017-09-13 09:34:00  
                                                Zug                    2017-09-14 09:34:00  
                                85:11:11033:001 Zug                    2017-09-13 09:55:00

# SCHEDULES

In [25]:
df_partial_schedules = schedules.partial_schedules(visu_trips.reset_index())

In [26]:
df_partial_schedules.head()

,origin,destination,product_id
0,Aigle,Bex,Zug
1,Aigle,Roche VD,Zug
2,Arnex,Croy-Romainmôtier,Zug
3,Arnex,La Sarraz,Zug
4,Arth-Goldau,Zug,Zug


In [28]:
df_schedules = schedules.schedules(info_trips.reset_index(), df_partial_schedules)

In [29]:
df_schedules.head()

,origin,origin_coord,destination,destination_coord,product_id,trip_id,date_id,scheduled_departure_time,scheduled_arrival_time,scheduled_travel_time,actual_departure_time,actual_arrival_time,actual_travel_time
0,Aigle,6.963680:46.316845,Bex,7.000677:46.251460,Zug,85:11:12289:001,0.0,2017-09-13 23:45:00,2017-09-13 23:50:00,00:05:00,NaT,NaT,NaT
1,Aigle,6.963680:46.316845,Bex,7.000677:46.251460,Zug,85:11:12289:001,1.0,2017-09-14 23:45:00,2017-09-14 23:50:00,00:05:00,NaT,NaT,NaT
2,Aigle,6.963680:46.316845,Roche VD,6.927049:46.358686,Zug,85:11:12212:001,0.0,2017-09-13 05:14:00,2017-09-13 05:17:00,00:03:00,2017-09-13 05:17:13,2017-09-13 05:20:03,00:02:50
3,Aigle,6.963680:46.316845,Roche VD,6.927049:46.358686,Zug,85:11:12212:001,1.0,2017-09-14 05:14:00,2017-09-14 05:17:00,00:03:00,2017-09-14 05:15:16,2017-09-14 05:18:06,00:02:50
8,Arnex,6.518903:46.698067,Croy-Romainmôtier,6.478119:46.696704,Zug,85:11:12212:001,0.0,2017-09-13 06:28:00,2017-09-13 06:33:00,00:05:00,2017-09-13 06:35:15,2017-09-13 06:36:15,00:01:00


In [30]:
df_schedules_viz = df_schedules
df_schedules_viz = df_schedules_viz.set_index(['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id'])
df_schedules_viz = df_schedules_viz[['scheduled_departure_time', 'scheduled_arrival_time', 'scheduled_travel_time',
                'actual_departure_time', 'actual_arrival_time', 'actual_travel_time']]

In [31]:
df_schedules_viz.head()

scheduled_departure_time  \
origin origin_coord       destination       destination_coord  product_id                            
Aigle  6.963680:46.316845 Bex               7.000677:46.251460 Zug             2017-09-13 23:45:00   
                                                               Zug             2017-09-14 23:45:00   
                          Roche VD          6.927049:46.358686 Zug             2017-09-13 05:14:00   
                                                               Zug             2017-09-14 05:14:00   
Arnex  6.518903:46.698067 Croy-Romainmôtier 6.478119:46.696704 Zug             2017-09-13 06:28:00   

                                                                          scheduled_arrival_time  \
origin origin_coord       destination       destination_coord  product_id                          
Aigle  6.963680:46.316845 Bex               7.000677:46.251460 Zug           2017-09-13 23:50:00   
                                                               Zug           2017-09-14 23:50:00   
                          Roche VD          6.927049:46.358686 Zug           2017-09-13 05:17:00   
                                                               Zug           2017-09-14 05:17:00   
Arnex  6.518903:46.698067 Croy-Romainmôtier 6.478119:46.696704 Zug           2017-09-13 06:33:00   

                                                                          scheduled_travel_time  \
origin origin_coord       destination       destination_coord  product_id                         
Aigle  6.963680:46.316845 Bex               7.000677:46.251460 Zug                     00:05:00   
                                                               Zug                     00:05:00   
                          Roche VD          6.927049:46.358686 Zug                     00:03:00   
                                                               Zug                     00:03:00   
Arnex  6.518903:46.698067 Croy-Romainmôtier 6.478119:46.696704 Zug                     00:05:00   

                                                                          actual_departure_time  \
origin origin_coord       destination       destination_coord  product_id                         
Aigle  6.963680:46.316845 Bex               7.000677:46.251460 Zug                          NaT   
                                                               Zug                          NaT   
                          Roche VD          6.927049:46.358686 Zug          2017-09-13 05:17:13   
                                                               Zug          2017-09-14 05:15:16   
Arnex  6.518903:46.698067 Croy-Romainmôtier 6.478119:46.696704 Zug          2017-09-13 06:35:15   

                                                                          actual_arrival_time  \
origin origin_coord       destination       destination_coord  product_id                       
Aigle  6.963680:46.316845 Bex               7.000677:46.251460 Zug                        NaT   
                                                               Zug                        NaT   
                          Roche VD          6.927049:46.358686 Zug        2017-09-13 05:20:03   
                                                               Zug        2017-09-14 05:18:06   
Arnex  6.518903:46.698067 Croy-Romainmôtier 6.478119:46.696704 Zug        2017-09-13 06:36:15   

                                                                          actual_travel_time  
origin origin_coord       destination       destination_coord  product_id                     
Aigle  6.963680:46.316845 Bex               7.000677:46.251460 Zug                       NaT  
                                                               Zug                       NaT  
                          Roche VD          6.927049:46.358686 Zug                  00:02:50  
                                                               Zug                  00:02:50  
Arnex  6.518903:46.6

# EDGES

In [32]:
df_edges = edges.get_edges_params(df_schedules, defined_timeslots)

In [33]:
df_edges.head()

,fromNode,fromCoord,toNode,toCoord,product_id,timeslots,"edge_params [µ, s]"
0,Aigle,6.963680:46.316845,Bex,7.000677:46.251460,Zug,00:00:00-12:00:00,"(nan, nan)"
1,Aigle,6.963680:46.316845,Bex,7.000677:46.251460,Zug,12:00:00-23:59:59,"(nan, nan)"
2,Aigle,6.963680:46.316845,Roche VD,6.927049:46.358686,Zug,00:00:00-12:00:00,"(-10.0, 0.0)"
3,Aigle,6.963680:46.316845,Roche VD,6.927049:46.358686,Zug,12:00:00-23:59:59,"(nan, nan)"
4,Arnex,6.518903:46.698067,Croy-Romainmôtier,6.478119:46.696704,Zug,00:00:00-12:00:00,"(-240.0, 0.0)"


# NODES

In [34]:
df_nodes = nodes.get_nodes_params(df_schedules, defined_timeslots)

In [35]:
df_nodes.head()

,node,coord,product_id,timeslot,"departure_params [µ, s]","arrival_params [µ, s]"
0,Aigle,6.963680:46.316845,Zug,00:00:00-12:00:00,"(134.5, 82.7)","(100.0, 84.9)"
1,Aigle,6.963680:46.316845,Zug,12:00:00-23:59:59,"(nan, nan)","(nan, nan)"
2,Arnex,6.518903:46.698067,Zug,00:00:00-12:00:00,"(230.3, 145.6)","(49.8, 108.5)"
3,Arnex,6.518903:46.698067,Zug,12:00:00-23:59:59,"(259.7, 166.2)","(18.1, 97.2)"
4,Arth-Goldau,8.549491:47.049157,Zug,00:00:00-12:00:00,"(71.5, 27.6)","(-19.0, 58.0)"


In [ ]:
# save dataframes
'''
df.to_pickle('df')
info_trips.to_pickle('info_trips')
visu_trips.to_pickle('visu_trips')
df_schedules.to_pickle('df_schedules')
df_edges.to_pickle('df_edges')
df_nodes.to_pickle('df_nodes')'''